# Part I Python Fundamentals 4

This is my review note of Python for the purpose of self-study. The note mixes up with English & Chinese.
- Part I follows the *[Liao's Python tutorial](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000) (in Chinese)*

In [3]:
# Display multiple interactive objects in one shell
# No Need for print function
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### 8.错误、调试和测试

错误
- 可预测错误
- 不可预测错误（异常）： 比如写入文件的时候，磁盘满了，写不进去了，或者从网络抓取数据，网络突然断掉了

In [10]:
# 出现错误就会跳转到 except, 然后执行 finally
try:
    print('try...')
    r = 10 / 0 # error
    print('result:', r) # 不会被执行，直接跳转到 except
except ZeroDivisionError as e:
    print('except:', e)
except ValueError as e:
    print('ValueError')
    
# 此外，如果没有错误发生，可以在except语句块后面加一个else，当没有错误发生时，会自动执行else语句：
else:
    print('no error!')

# 无论有没有 error, finally 都会被执行
finally:
    print('finally...')

try...
except: division by zero
finally...


ython的错误其实也是class，所有的错误类型都继承自BaseException，所以在使用except时需要注意的是，它不但捕获该类型的错误，还把其子类也“一网打尽”

logging
- 记录错误，并且继续运行到底

In [18]:
import logging

try:
    x = 10 / 0 # 出错
except ZeroDivisionError as e:
    logging.exception(e) # 记录错误，继续运行
    
print('end')

end


抛出错误